# 소실점

In [2]:
import cv2
import numpy as np

def grayscale(img): # 흑백이미지로 변환
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold): # Canny알고리즘
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kerner_size): # 가우시안 필터
    return cv2.GaussianBlur(img, (kerner_size, kerner_size), 0)

def region_of_interest(img, vertices, color3 = (255, 255, 255), color1 = 255):
    #ROI setting
    mask = np.zeros_like(img) # img와 같은 크기의 빈 이미지

    if len(img.shape) > 2 : #if color image (3 channels)
        color = color3
    else : #if black&white image ( 1 channel)
        color = color1
    
    #vertices에 정한 점들로 이루어진 다각형 부분(ROI부분)을 color로 채움
    cv2.fillPoly(mask, vertices, color)
    
    #이미지와 color로 체워진 ROI를 합침
    ROI_img = cv2.bitwise_and(img, mask)
    return ROI_img

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap): #허프 변환
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype = np.uint8)
    #draw_lines(line_img, lines)
    
    return lines

def weighted_img(img, initial_img, a = 1, b = 1., c = 0.): #두 이지미 합치기
    return cv2.addWeighted(initial_img, a, img, b, c)


In [3]:
def get_fitline(img, f_lines):
    lines = np.squeeze(f_lines)
    lines = lines.reshape(lines.shape[0] * 2, 2)
    rows, cols = img.shape[:2]
    output = cv2.fitLine(lines, cv2.DIST_L2, 0, 0.01, 0.01)
    
    vx, vy, x, y = output[0], output[1], output[2], output[3]
    x1, y1 = int(((img.shape[0] - 1) - y) / vy * vx + x), img.shape[0] - 1
    x2, y2 = int(((img.shape[0]/2 + 100) - y) / vy * vx + x), int(img.shape[0] / 2 + 100)
    
    result = [x1,y1, x2, y2]
    return result

def draw_lines(img, lines, color=[0,0,255], thickness = 10): #선그리기
    cv2.line(img, (lines[0], lines[1]), (lines[2], lines[3]), color, thickness)
    
#소실점
def draw_cross_dot(line1, line2, img, color = (0, 0, 255), thickness = 10):
    x1, y1, x2, y2 = line1[0], line1[1], line1[2], line1[3]
    x3, y3, x4, y4 = line2[0], line2[1], line2[2], line2[3]

    xc = int(((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / ((x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)))
    yc = int(((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / ((x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)))

    cv2.line(img, (xc, yc), (xc, yc), color, thickness)
    
    return (xc, yc)
    
def draw_center_dot(img, color = (0, 0, 0), thickness = 10):
    height, width = img.shape[0], img.shape[1]
    cv2.line(img, (width//2, 0), (width//2, height), color, thickness)
    
def leftorright(img, xc, yc, allow = 10):
    Xcenter, Ycenter = img.shape[1] // 2, img.shape[0] // 2
    
    if Xcenter - allow < xc and xc < Xcenter + allow :
        print("직진")
    elif xc <= Xcenter - allow:
        print("좌회전")
    elif xc >= Xcenter + allow:
        print("우회전")

In [4]:
cap_test = cv2.VideoCapture("test/RightC.mp4")

_, imga = cap_test.read()
draw_center_dot(imga)
print([imga.shape[0], imga.shape[1]])

[1440, 2560]


# 동영상 적용

In [8]:

cap1 = cv2.VideoCapture("test/RightC.mp4")
cap2 = cv2.VideoCapture("test/solidWhiteRight.mp4")

while (cap1.isOpened()):
    ret, image = cap1.read()
    if not ret:
        break
    height, width = image.shape[:2]

    gray_img = grayscale(image) # 흑백이미지로 변환

    blur_img = gaussian_blur(gray_img, 3) # Blur 효과

    canny_img = canny(blur_img, 70, 210) # Canny Edge 알고리ㅑ

    #cv2.imshow('canny', canny_img) # Canny 이미지 출력

    vertices = np.array([[(300, height), (width/2 - 45, height/2 + 300), (width/2 + 45, height/2 + 300), (width-200, height)]], dtype=np.int32)

    roi_img = region_of_interest(canny_img, vertices) # vertices에 정한 점들을 기준으로 ROI이미지 생성
    #cv2.imshow('roi', roi_img)
    
    line_arr = hough_lines(roi_img, 1, 1 * np.pi/180, 30, 10 ,20) #허프 변환
    line_arr = np.squeeze(line_arr)

    #기울기 구하기
    slope_degree = (np.arctan2(line_arr[:,1] - line_arr[:,3], line_arr[:,0] - line_arr[:, 2]) * 180) / np.pi

    #수평 기울기 제한
    line_arr = line_arr[np.abs(slope_degree) < 160]
    slope_degree = slope_degree[np.abs(slope_degree) < 160]

    #수직 기울기 제한
    line_arr = line_arr[np.abs(slope_degree) > 95]
    slope_degree = slope_degree[np.abs(slope_degree) > 95]

    #필터링 직선 버리기
    L_lines, R_lines = line_arr[(slope_degree > 0), :], line_arr[(slope_degree < 0), :]
    temp = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    L_lines, R_lines = L_lines[:,None], R_lines[:,None]

    #왼쪽 오른쪽 각각 대표선 구하기
    left_fit_line = get_fitline(image, L_lines)
    right_fit_line = get_fitline(image, R_lines)

    #대표선 그리기
    draw_lines(temp, left_fit_line)
    draw_lines(temp, right_fit_line)

    #중앙점, 소실점 찍기
    draw_center_dot(temp, color=(255,0,0))
    xc, yc = draw_cross_dot(left_fit_line, right_fit_line, temp)
    
    leftorright(temp, xc, yc, image.shape[1] / 70)
    
    result = weighted_img(temp, image)
    cv2.imshow('res', result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release
cap1.release()
cv2.destroyAllWindows()


직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
좌회전
직진
직진
좌회전
직진
직진
직진
직진
좌회전
좌회전
직진
직진
직진
좌회전
직진
직진
좌회전
직진
좌회전
직진
직진
좌회전
직진
직진
직진
직진
좌회전
직진
직진
직진
직진
직진
직진
직진
좌회전
직진
좌회전
직진
우회전
우회전
직진
우회전
우회전
직진
우회전
우회전
우회전
우회전
우회전
좌회전
직진
좌회전
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
우회전
직진
직진
직진
우회전
우회전
직진
직진
직진
직진
우회전
직진
직진
직진
우회전
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
우회전
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
직진
우회전
직진
직진
직진
직진
직진
직진
직진